[View in Colaboratory](https://colab.research.google.com/github/tcglarry/aoi/blob/master/aoi_densenet_testing_start.ipynb)

In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18408 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8.1_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8.1) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [3]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
!mkdir -p drive
!google-drive-ocamlfuse drive
!ls drive

aoi_aoi_test1.ipynb  PiXNet				      test for excel
aoi_test	     Sample upload.txt			      Untitled0.ipynb
Capstone_PixNet.zip  TAHU PowerPoint Template.pdf	      Untitled1.ipynb
Colab Notebooks      TAHU PowerPoint Template - 柱狀圖 1.ods  Untitled2.ipynb


In [2]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image

from keras.layers import Dense, GlobalAveragePooling2D


import cv2
import numpy as np
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import subprocess
import os
import pickle
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate
from keras.models import Model, load_model
import pandas as pd
import sklearn 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from PIL import Image
import time 
import pandas as pd

Using TensorFlow backend.


In [4]:
src='drive/aoi_test/'


df_test= pd.read_csv(src+'test.csv')
test_img = df_test['ID'].values  
print ('iamge list size', len(test_img))

iamge list size 10142


In [0]:
def testing(test_data, model):
    start = time.time()
    test_pred = model.predict(test_data)
    print (test_pred.shape)
    test_pred=np.squeeze(test_pred)

    print (test_pred.shape)

    test_pred= np.argmax(test_pred, axis=1)
    print (test_pred.shape)

    end = time.time()
    print ('total time = ', round(end - start,4))
    
    return test_pred

In [6]:
df_final = pd.DataFrame()
df_dict= {}

if os.path.isfile(src+'densenet.h5'):
    
    
    model = load_model(src+'densenet.h5')
    print ('model laoded')
else:
    print ('model not exist')

model laoded


In [0]:
t_s = time.time()

for i in range(102):
    s= time.time()
    
    img_list=[]
    
    if i < 101 : 
        test_list = test_img[i*100 : (i+1)*100]
    else:
        test_list = test_img[i*100 : ]
    df_dict['ID'] = test_list
    for i,img in enumerate(test_list):
        x = cv2.imread(src+'test_images/'+img)
        x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)


        img_list.append(x)
    
    test_data = np.array(img_list)
    print ('\n*****start inferencing*****\n')
    result = testing (test_data, model) 
    df_dict['Label'] = result
    df_test = pd.DataFrame(df_dict)
    df_final = df_final.append(df_test, ignore_index = True)
    #print (df_final[(i+1)*100 - 5: (i+1)*100 + 5])
    e= time.time()
    print 
    print ('batch', i, ' time needed = ', round(e - s,4))
    
print (len(df_final))
t_e = time.time()
print ( ' total time needed = ', round(t_e - t_s,4))
           


*****start inferencing*****



In [0]:
df_final.to_csv ('test2.csv')

In [0]:
test_data = np.array(img_list)
print (train_data.shape)
#train_data =  train_data[:,:,:,np.newaxis]
print (test_data.shape)

In [0]:
df_test= pd.read_csv('test.csv')
df_test.head()
df_test.keys()    

Index(['ID', 'Label'], dtype='object')

In [0]:
df_y = df_test[:20] 

In [0]:
df_y ['Label']= test_pred

/Users/LarryGuo/miniconda3/envs/inpainting-style/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [0]:
df_y

,ID,Label
0,test_00000.png,0
1,test_00001.png,3
2,test_00002.png,3
3,test_00003.png,3
4,test_00004.png,0
5,test_00005.png,3
6,test_00006.png,3
7,test_00007.png,0
8,test_00008.png,0
9,test_00009.png,3


In [0]:
df_y.to_csv('test_2.csv')

In [0]:
for i in range (len(test_img)//):
    

In [0]:
df_test.append(df_test, ignore_index=True)

In [0]:
df_test